In [ ]:
# Extraction of timedate when it was published

In [ ]:
#!/usr/bin/env python
import requests
import json
import pandas as pd
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

base = 'https://dataverse.harvard.edu'
rows = 1000
start = 0
page = 1
condition = True # emulate do-while
prueba = requests.Session()
retry = Retry(connect=3, backoff_factor=2)
adapter = HTTPAdapter(max_retries=retry)
prueba.mount('http://', adapter)
prueba.mount('https://', adapter)
scraping_types=pd.DataFrame(columns=['Type'])
scraping_names=pd.DataFrame(columns=['Name'])
scraping_publication=pd.DataFrame(columns=['PublicationDate'])



while (condition):
    url = base + '/api/search?q=*&type=file&show_facets=true&per_page=1000' + "&start=" + str(start)
    data = json.loads(prueba.get(url).text)
    total = data['data']['total_count']
    print("=== Page", page, "===")
    print("start:", start, " total:", total)
    for i in data['data']['items']:
        scraping_types=scraping_types.append(pd.DataFrame([i['type']], columns=['Type']))
        scraping_names=scraping_names.append(pd.DataFrame([i['name']], columns=['Name']))
        scraping_publication=scraping_publication.append(pd.DataFrame([i['published_at']], columns=['PublicationDate']))

    start = start + rows
    page += 1
    condition = start < total/5

=== Page 1 ===
start: 0  total: 418824
=== Page 2 ===
start: 1000  total: 418824
=== Page 3 ===
start: 2000  total: 418824
=== Page 4 ===
start: 3000  total: 418824
=== Page 5 ===
start: 4000  total: 418824


# Extraction of file formats

In [84]:
#!/usr/bin/env python
import requests
import json
import pandas as pd
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

base = 'https://dataverse.harvard.edu'
rows = 10
start = 0
page = 1
condition = True # emulate do-while
prueba = requests.Session()
retry = Retry(connect=3, backoff_factor=2)
adapter = HTTPAdapter(max_retries=retry)
prueba.mount('http://', adapter)
prueba.mount('https://', adapter)
filetypes_count=pd.DataFrame(columns=['Count'])
filetypes_name=pd.DataFrame(columns=['Name'])

url = base + '/api/search?q=*&type=file&show_facets=true&per_page=1' + "&start=" + str(start)
data = json.loads(prueba.get(url).text)
total = data['data']['total_count']
print("=== Page", page, "===")
print("start:", start, " total:", total)
    

for i in data['data']['facets']:
    for d in i['fileTypeGroupFacet']['labels']:
        for j in d:
            filetypes_count=filetypes_count.append(pd.DataFrame([d[j]], columns=['Count']))
            filetypes_name=filetypes_name.append(pd.DataFrame([j], columns=['Name']))

final=pd.concat([filetypes_name,filetypes_count],axis=1).reset_index()
print(final)

=== Page 1 ===
start: 0  total: 418822
    index          Name   Count
0       0          Data  114481
1       0       Unknown   71496
2       0      Document   59259
3       0  Tabular Data   47471
4       0          Text   44846
5       0         Image   40509
6       0   Application   23623
7       0           ZIP    5862
8       0          FITS    4995
9       0         Audio    3812
10      0         Video    1136
11      0         Shape     868
12      0          text     245
13      0          fits     144
14      0  Network Data      55
15      0      Chemical       9
16      0        Binary       7
17      0         Model       2
18      0      document       1
19      0       unknown       1


In [ ]:
final.csv('fileformats.csv',sep=";", encoding="UTF-16")